In [ ]:
from datasets import load_dataset
import numpy as np

# Cargar dataset
dataset = load_dataset("cosimoiaia/RTTTL-Ringtones", split="train")

# Extraer solo la parte de la melodía RTTTL
def extract_notes(rtttl_string):
    try:
        parts = rtttl_string.split(":")
        notes = parts[2].split(",")
        return [note.strip() for note in notes]
    except:
        return []

melodies = [extract_notes(x["text"]) for x in dataset]
melodies = [m for m in melodies if len(m) >= 20]  # filtrar secuencias muy cortas

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
print(melodies[0][:30])


['16c.5', '32d.5', '32d.5', '16d.5', '8d.5', '16c.5', '16c.5', '16c.5', '16c.5', '32d#.5', '32d#.5', '16d#.5', '8d#.5', '16d.5', '16d.5', '16d.5', '16c.5', '32d.5', '32d.5', '16d.5', '8d.5', '16c.5', '16c.5', '16c.5', '16c.5', '32d#.5', '32d#.5', '16d#.5', '8d#.5', '16d.5']


In [6]:
all_notes = [note for melody in melodies for note in melody]
unique_notes = sorted(set(all_notes))
note2idx = {note: i for i, note in enumerate(unique_notes)}
idx2note = {i: note for note, i in note2idx.items()}
vocab_size = len(unique_notes)

print(f"Vocabulario tamaño: {vocab_size}")


Vocabulario tamaño: 1403


In [7]:
sequence_length = 20
X = []
y = []

for melody in melodies:
    encoded = [note2idx[note] for note in melody]
    for i in range(len(encoded) - sequence_length):
        X.append(encoded[i:i+sequence_length])
        y.append(encoded[i+sequence_length])

X = np.array(X)
y = np.array(y)

print(f"Cantidad de secuencias para entrenamiento: {X.shape[0]}")


Cantidad de secuencias para entrenamiento: 309490


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=sequence_length),
    LSTM(128),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, sequence_length))
model.summary()

c:\Users\gerar\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 20, 64)         │        89,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1403)           │       180,987 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 369,595 (1.41 MB)

 Trainable params: 369,595 (1.41 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.fit(X, y, epochs=20, batch_size=64, validation_split=0.1)

Epoch 1/20
4353/4353 ━━━━━━━━━━━━━━━━━━━━ 101s 23ms/step - accuracy: 0.0876 - loss: 4.8211 - val_accuracy: 0.1683 - val_loss: 3.6526
Epoch 2/20
4353/4353 ━━━━━━━━━━━━━━━━━━━━ 102s 23ms/step - accuracy: 0.1950 - loss: 3.4220 - val_accuracy: 0.1945 - val_loss: 3.4172
Epoch 3/20
4353/4353 ━━━━━━━━━━━━━━━━━━━━ 97s 22ms/step - accuracy: 0.2213 - loss: 3.1594 - val_accuracy: 0.2013 - val_loss: 3.3444
Epoch 4/20
4353/4353 ━━━━━━━━━━━━━━━━━━━━ 96s 22ms/step - accuracy: 0.2443 - loss: 3.0028 - val_accuracy: 0.2077 - val_loss: 3.3029
Epoch 5/20
4353/4353 ━━━━━━━━━━━━━━━━━━━━ 100s 23ms/step - accuracy: 0.2588 - loss: 2.8993 - val_accuracy: 0.2128 - val_loss: 3.2849
Epoch 6/20
4353/4353 ━━━━━━━━━━━━━━━━━━━━ 95s 22ms/step - accuracy: 0.2759 - loss: 2.8091 - val_accuracy: 0.2108 - val_loss: 3.2879
Epoch 7/20
4353/4353 ━━━━━━━━━━━━━━━━━━━━ 96s 22ms/step - accuracy: 0.2900 - loss: 2.7370 - val_accuracy: 0.2139 - val_loss: 3.2952
Epoch 8/20
4353/4353 ━━━━━━━━━━━━━━━━━━━━ 96s 22ms/step - accuracy: 0.300

In [11]:
model.save("rtttl_ringtone_model.h5")

In [30]:
import random

def sample_with_temperature(preds, temperature=1.0):
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_notes(model, seed_seq, length=50, temperature=1.0):
    generated = seed_seq[:]
    for _ in range(length):
        input_seq = np.array(generated[-sequence_length:]).reshape(1, sequence_length)
        preds = model.predict(input_seq, verbose=0)[0]
        next_idx = sample_with_temperature(preds, temperature)
        generated.append(next_idx)
    return [idx2note[i] for i in generated]

In [31]:
custom_seed_notes = ["4c5", "4f5", "4g5", "4c6", "4g5", "4f5", "4c5"]
seed_seq = [note2idx[note] for note in custom_seed_notes]
# Asegurar que tenga la longitud adecuada
while len(seed_seq) < sequence_length:
    seed_seq.insert(0, 0)  # padding con índice 0

new_notes = generate_notes(model, seed_seq, length=200, temperature=0.8)

rtttl_string = "Generada:d=4,o=5,b=140:" + ",".join(new_notes)
print("RTTTL generado:")
print(rtttl_string)

RTTTL generado:
Generada:d=4,o=5,b=140:,,,,,,,,,,,,,4c5,4f5,4g5,4c6,4g5,4f5,4c5,d,c,c,a#5,4p.,d,d#,d,d#,4g,d#,d,c,d#,d,c#,4d,4c,p,c,a#5,c,d,d,d#,d,c,4d,f,d#,d,c,d,c,4a5,c,c,c,c,c,2a#5,d,d,d,d,c,d,d,c,d,d,c,c,4d,c,d,4c,4d,4d,d,d,d,2d,2p,d,d,d,d,c,4d,4d.,2p,c,d,d,4d.,p,d,d,d,1c,4p,d,d,d,d,d,d,d,d,d,4d,f,d,d,d,d,4d.,p,d,e,d,d,d,4d.,p,d,4f.,p,d,d,4c.,p,d,d,4d.,p,d,d,d,d,d,d,4d.,p,d,d,a,4g,f,4g,f,d,d,c,d,f,d,d,4c,d,d,d,d,d,d,4c,d,e,f,g,f,e,f,f,d,d,d,c,e,f,f,e,f,f,d,c,c,d,c,d,d,d,c,d,d,d,c,d,e,f,f,f,f,e,f,f,f,f,g,f,e,d,e,e,2f,e,f,e,1f


In [16]:
!pip install mido python-rtmidi



In [ ]:
from mido import MidiFile, MidiTrack, Message, MetaMessage, bpm2tempo

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

# Definir tempo
bpm = 120
tempo = bpm2tempo(bpm)
track.append(MetaMessage('set_tempo', tempo=tempo))

ticks_per_beat = mid.ticks_per_beat
beats_per_second = bpm / 60
max_ticks = int(ticks_per_beat * beats_per_second * 20)  # 20 segundos

total_ticks = 0

for note_str in new_notes:
    parsed = parse_note(note_str)
    if not parsed:
        continue
    dur, midi_note = parsed
    ticks = int(ticks_per_beat * (4 / dur))  # calcula duración real de la nota

    if total_ticks + ticks > max_ticks:
        break

    track.append(Message('note_on', note=midi_note, velocity=64, time=0))
    track.append(Message('note_off', note=midi_note, velocity=64, time=ticks))
    track.append(Message('program_change', program=24, time=0))  # Nylon Guitar

    total_ticks += ticks

mid.save("output_corrido.mid")
print("✅ Corrido generado y guardado como output_corrido.mid (20 segundos)")


✅ Corrido generado y guardado como output_corrido.mid (20 segundos)
